# Installation
Simply run:

    pip install git+https://github.com/Clopeq/PyPep3.git

Then simply import


In [1]:
import PyPep3 as pep

# Solution

First a Solution object has to be created. This object will be responsible for all of the calculations.

Solution object makes use of "*input file*". This is the file containg all the relevant thermodynamic data. Just like Propep uses it's own input files which you can edit so does the PyPep3

Solution object by default uses (TODO: which file?) input file. Alternatively input file can be specified manually.

*Note: Never directly edit the input files. Make sure you create backup first*

In [2]:

solution = pep.Solution()               # The default input file will be used
solution = pep.Solution("gri30.yaml")   # Custom input file

# Reactants

Usually there will be two reactants: fuel and oxidizer.

However any number of reactants can be created.

The reactants has to be initialized first as Phase objects

In [3]:
fuel1 = pep.Phase(solution)  # the solution object has to be passed as an argument
fuel2 = pep.Phase(solution)  # there may be more than 2 reactants
ox = pep.Phase(solution)        

## Reactant composition

The chemical composition of each reactant is set with .comp attribute. Either by setting it as dictionary of species-fraction or as single string (look example below).

*Note: Sum off all the fraction does not need to sum up to 1*

In [4]:
fuel1.comp = {'H2':0.2, "N2":0.8}       # mixture of 20% hydrogen, 80% Nitrogen
fuel2.comp = 'H2:2, CH4:5'              # mixture of 2/7 th hydrogen and 5/7 th methane
ox.comp = "O2:1"                        # pure oxygen

print(f"fuel1: {fuel1.comp}")
print(f"fuel2: {fuel2.comp}")
print(f"ox: {ox.comp}")

fuel1: {'H2': 0.2, 'N2': 0.8}
fuel2: {'CH4': 0.7142857142857142, 'H2': 0.2857142857142857}
ox: {'O2': 1.0}


## Reactant thermodynamic state

The state of reactants can be set individually with .TP property

In [5]:
fuel1.TP = 300, 10e5    # fuel1 is at 300 K and 10 bar (10e5 Pa) when it enters the reactor (combustion chamber)
fuel2.TP = 250, 20e5    # 250 K, 20 bar
ox.TP    = 200, 5e5     # 200 K, 5 bar

print("fuel1 temperature: %.1f K" % fuel1.T)
print("fuel2 temperature: %.1f K" % fuel2.T)
print("ox temperature: %.1f K" % ox.T)
print("fuel1 pressure: %d Pa" % fuel1.P)
print("fuel2 pressure: %d Pa" % fuel2.P)
print("fuoxel1 pressure: %d Pa" % ox.P)

fuel1 temperature: 300.0 K
fuel2 temperature: 250.0 K
ox temperature: 200.0 K
fuel1 pressure: 1000000 Pa
fuel2 pressure: 1999999 Pa
fuoxel1 pressure: 500000 Pa


## Modifing the state


Temperature and pressure can be set independently by passing None as one parameter


This is useful for calculations where the propellant state is not static and has some changes
For example nitrous oxide in blow down hybrid. N2O temperature and pressure will change during the burn-time significantly

In [6]:

fuel1.TP = None, 15e5   # Do not change temperature, set pressure to 15 bar
fuel2.TP = 275, None    # Set temperature to 275 K, pressure does not change


print("fuel1 temperature: %.1f K" % fuel1.T)
print("fuel2 temperature: %.1f K" % fuel2.T)
print("fuel1 pressure: %d Pa" % fuel1.P)
print("fuel2 pressure: %d Pa" % fuel2.P)

fuel1 temperature: 300.0 K
fuel2 temperature: 275.0 K
fuel1 pressure: 1500000 Pa
fuel2 pressure: 1999999 Pa


# Mixing

Evry Phase object can be mixed together to create more complex propellant mixtures by simply summing this objects together.
The new temperature and pressure will be calculated automatically.
The totl internal energy of both fuels is computed and then divided by their combined mass. It does the same with volumes.
Other parameters like temperature and pressure (and enthalpy?) are calculated based on this new state
It is possible to mix fuels with oxidizers, however this is not advised

In [7]:

fuel = fuel1 + fuel2 # This will create new fuel mixture with 50% fuel1 and 50% fuel2.
# This is becouse 1 kg (default value, more about it later) of each is mixed to produce 2 kg of new mixture

print(f"fuel: {fuel.comp}")
print("fuel temperature: %.1f K" % fuel.T)
print("fuel pressure: %d Pa" % fuel.P)

fuel: {'CH4': 0.3571428571428571, 'H2': 0.24285714285714285, 'N2': 0.4}
fuel temperature: 284.8 K
fuel pressure: 1747929 Pa


# Oxidizer to fuel ratio

By default each Phase object has a total mass of 1kg
The amount of mass isn't important, what matters is the fraction of each reactant mass in the total propellant mixture

Just like in ProPep one would assign 80g of N2O and 10g of ABS to set the OF ratio to 8

in PyPep3 this can be done in one of two ways
Simply update the reactant masses, or use Solution.set_of_ratio() method

In [8]:
ox.mass = 5     # 5 kg of oxidizer
fuel.mass = 1   # 1 kg of fuel

solution.set_of_ratio(fuel, ox, 5)  # this sets the fuel mass to 1 kg and ox mass to 5 kg

print("ox.mass: %.2f kg" % ox.mass)
print("fuel.mass: %.2f kg" % fuel.mass)

ox.mass: 5.00 kg
fuel.mass: 1.00 kg


# Solving the equilibrium

Solution.solve() method is used to calculate the equilibrium (burn the propellants). This method returns it's products as new Phase object (the same as reactants). So all of the same attributes apply. But now geting the value of this attributes is of intrest less than setting them to specific values

In [9]:

gas = solution.solve(fuel, ox) # gas containts reaction products

print("---------- COMBUSTION PRODUCTS -------------------")
print(f"Composition of exhaust gasses: {gas.comp}")    # returns the compostionion of the products (a dictionary of species: mass fraction pairs)
print("Adiabatic flame temperature: %.1f K" % gas.T) 
print("Combustion pressure: %d Pa" % gas.P)
print("Specific heat ratio: %.3f" % gas.k) 
print("Mean molecular weight: %.2f g/mol" % gas.M)    
print("Ideal characteristic velocity: %.1f m/s" % gas.cstar)  
print("Specific gas constant: %2.f J/kg-K" % gas.Rs)     
print("Specific heat at constant pressure: %.1f J/kg-K" % gas.cp)  
print("Specific heat at constant volume: %.1f J/kg-K" % gas.cv)   
print()

---------- COMBUSTION PRODUCTS -------------------
Composition of exhaust gasses: {'C': 1.9225782567807883e-12, 'C2H': 5.114059653338369e-18, 'C2H2': 4.675624631770057e-17, 'C2H3': 9.298709119177069e-21, 'C2H4': 1.585567519793641e-21, 'C2H5': 4.591281635507991e-25, 'C2H6': 1.1538106227845776e-26, 'C3H7': 2.100815407174924e-37, 'C3H8': 4.822707646860352e-39, 'CH': 3.7411795224806204e-13, 'CH2': 3.2102791448659986e-13, 'CH2(S)': 3.5598979461307936e-14, 'CH2CHO': 1.2861641902826223e-19, 'CH2CO': 1.0900941152620375e-15, 'CH2O': 4.614102965796884e-09, 'CH2OH': 6.342569753917131e-13, 'CH3': 3.5676855418334546e-13, 'CH3CHO': 1.0457860986671274e-20, 'CH3O': 1.735789159613965e-14, 'CH3OH': 2.221835142126594e-14, 'CH4': 4.976139161648396e-14, 'CN': 8.937603982207591e-11, 'CO': 0.035778214049453624, 'CO2': 0.1070702300888065, 'H': 0.0008106741838131093, 'H2': 0.0031664944708422304, 'H2CN': 2.7562311442537505e-14, 'H2O': 0.4188600537146512, 'H2O2': 2.2165042415405015e-05, 'HCCO': 3.451447823168063

# Minimal example

In [10]:
# Minimal example, calculating adiabatic flame temperature
import PyPep3 as pep

# initialization
solution = pep.Solution()
fuel = pep.Phase(solution)
ox = pep.Phase(solution)

# Set the propellant composition and state
fuel.comp = "H2:1"
ox.comp = "O2:1"
fuel.TP = 100, 40e5
ox.TP = 170, 42e5

# Burn the propellants adiabatically (constant enthalpy and pressure)
gas = solution.solve(fuel, 5*ox) # 5*ox <- effectively set the OF ratio to 5
print("Adiabatic flame temperature: %d K" % gas.T)

Adiabatic flame temperature: 3316 K


In [12]:
import PyPep3 as pep

# Create solution (thermodynamic model)
solution = pep.Solution()

# Define reactants
fuel = pep.Phase(solution)
ox   = pep.Phase(solution)

# Set composition (mass-based)
fuel.comp = "H2:1"
ox.comp   = "O2:1"

# Set thermodynamic state (T [K], P [Pa])
fuel.TP = 100, 40e5
ox.TP   = 170, 42e5

# Burn with O/F = 5
gas = solution.solve(fuel, 5 * ox)

print(f"Adiabatic flame temperature: {gas.T:.1f} K")
print(f"Mean molecular weight: {gas.M:.2f} g/mol")
print(f"Specific heat ratio (gamma): {gas.k:.3f}")


Adiabatic flame temperature: 3316.5 K
Mean molecular weight: 11.71 g/mol
Specific heat ratio (gamma): 1.206
